In [1]:
import re
import yaml
import os
import sys

In [2]:
sys.path.insert(0, '/home/jgutman/mvesc/ETL')
sys.path.insert(0, '/home/jgutman/mvesc/Models_Results')

In [3]:
from my_timer import Timer
from mvesc_utility_functions import *

In [4]:
def parse_addresses(addresses, shortened):
    addresses = [address.title() if address else None
        for address in addresses]
    addresses = [re.sub('[.,]', '', address) if address else None
        for address in addresses]
    addresses = [re.sub(r'\s+', ' ', address) if address else None
        for address in addresses]
    pattern = re.compile('|'.join(shortened.keys()))
    addresses = [pattern.sub(lambda x: shortened[x.group()], address)
        if address else None
        for address in addresses]
    return addresses

In [5]:
with open('/home/jgutman/mvesc/ETL/clean_addresses.yaml', 'r') as f:
    shortened = yaml.load(f)

In [44]:
with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        query_start_over = """drop table if exists all_snapshots;
            drop table if exists snapshots_updated;
            create table all_snapshots as 
            select * from clean.all_snapshots"""
        query_create_street_index = """create index old_streets on all_snapshots (street)"""
        cursor.execute(query_start_over)
        cursor.execute(query_create_street_index)
        query_get_addresses = """select street from all_snapshots"""
        cursor.execute(query_get_addresses)
        addresses = [i[0] for i in cursor.fetchall()]
        
        with Timer('parse_address') as t:
            standardized_addresses = parse_addresses(addresses, shortened)
            assert(len(standardized_addresses)==len(addresses))
            assert([i for i,j in enumerate(addresses) if not j] ==
                [i for i,j in enumerate(standardized_addresses) if not j])
            
        map_addresses = {k: v for k, v in zip(addresses, standardized_addresses) if k!=v}
        rows = [{'old_address': k, 'new_address': v} for k,v in map_addresses.items()]
        
        with Timer('replace_address') as t:
            cursor.execute("""alter table all_snapshots
                        add column street_clean_bare varchar(80)""")
            print(len(map_addresses.keys()))
            cursor.executemany("""update all_snapshots
                set street_clean_bare=%(new_address)s
                where street=%(old_address)s
                """, rows)
            cursor.execute("""create table snapshots_updated as
                select *, coalesce(street_clean_bare, street) street_clean
                from all_snapshots;
                alter table snapshots_updated
                drop column street_clean_bare""")
            cursor.execute("""select street_clean from snapshots_updated 
                           where street_clean is not null""")
            results = cursor.fetchall()
            print(len(results))
        connection.commit()

parse_address: 2.6 seconds elapsed
15233
243948
replace_address: 1.3e+01 seconds elapsed
